__Probing Language Models__

This notebook serves as a start for your NLP2 assignment on probing Language Models. This notebook will become part of the contents that you will submit at the end, so make sure to keep your code (somewhat) clean :-)

__note__: This assignment is not dependent on big fancy GPUs. I run all this stuff on my own 3 year old CPU, without any Colab hassle. So it's up to you to decide how you want to run it.

# Models

For the Transformer models you are advised to make use of the `transformers` library of Huggingface: https://github.com/huggingface/transformers
Their library is well documented, and they provide great tools to easily load in pre-trained models.

In [1]:
#
## Your code for initializing the transformer model(s)
#
# Note that most transformer models use their own `tokenizer`, that should be loaded in as well.
#
from transformers import GPT2Tokenizer, GPT2LMHeadModel

tokenizer = GPT2Tokenizer.from_pretrained('distilgpt2')
model = GPT2LMHeadModel.from_pretrained('distilgpt2')

In [2]:
#
## Your code for initializing the rnn model(s)
#
# The Gulordava LSTM model can be found here: 
# https://drive.google.com/file/d/19Lp3AM4NEPycp_IBgoHfLc_V456pmUom/view?usp=sharing
# You can read more about this model in the original paper here: https://arxiv.org/pdf/1803.11138.pdf
#
# N.B: I have altered the RNNModel code to only output the hidden states that you are interested in.
# If you want to do more experiments with this model you could have a look at the original code here:
# https://github.com/facebookresearch/colorlessgreenRNNs/blob/master/src/language_models/model.py
#
from collections import defaultdict
from lstm.model import RNNModel
import torch


model_location = 'state_dict.pt'  # <- point this to the location of the Gulordava .pt file
lstm = RNNModel('LSTM', 50001, 650, 650, 2)
lstm.load_state_dict(torch.load(model_location))


# This LSTM does not use a Tokenizer like the Transformers, but a Vocab dictionary that maps a token to an id.
with open('lstm/vocab.txt', encoding="utf8") as f:
    w2i = {w.strip(): i for i, w in enumerate(f)}

vocab = defaultdict(lambda: w2i["<unk>"])
vocab.update(w2i)

It is a good idea that before you move on, you try to feed some text to your LMs; and check if everything works accordingly. 

# Data

For this assignment you will train your probes on __treebank__ corpora. A treebank is a corpus that has been *parsed*, and stored in a representation that allows the parse tree to be recovered. Next to a parse tree, treebanks also often contain information about part-of-speech tags, which is exactly what we are after now.

The treebank you will use for now is part of the Universal Dependencies project. I provide a sample of this treebank as well, so you can test your setup on that before moving on to larger amounts of data.

Make sure you accustom yourself to the format that is created by the `conllu` library that parses the treebank files before moving on. For example, make sure you understand how you can access the pos tag of a token, or how to cope with the tree structure that is formed using the `to_tree()` functionality.

In [3]:
# READ DATA
from typing import List
from conllu import parse_incr, TokenList


# If stuff like `: str` and `-> ..` seems scary, fear not! 
# These are type hints that help you to understand what kind of argument and output is expected.
def parse_corpus(filename: str) -> List[TokenList]:
    data_file = open(filename, encoding="utf-8")

    ud_parses = list(parse_incr(data_file))
    
    return ud_parses

In [4]:
ud_parses = parse_corpus("data/en_ewt-ud-train.conllu")
print(ud_parses[:10])

[TokenList<Al, -, Zaman, :, American, forces, killed, Shaikh, Abdullah, al, -, Ani, ,, the, preacher, at, the, mosque, in, the, town, of, Qaim, ,, near, the, Syrian, border, .>, TokenList<[, This, killing, of, a, respected, cleric, will, be, causing, us, trouble, for, years, to, come, ., ]>, TokenList<DPA, :, Iraqi, authorities, announced, that, they, had, busted, up, 3, terrorist, cells, operating, in, Baghdad, .>, TokenList<Two, of, them, were, being, run, by, 2, officials, of, the, Ministry, of, the, Interior, !>, TokenList<The, MoI, in, Iraq, is, equivalent, to, the, US, FBI, ,, so, this, would, be, like, having, J., Edgar, Hoover, unwittingly, employ, at, a, high, level, members, of, the, Weathermen, bombers, back, in, the, 1960s, .>, TokenList<The, third, was, being, run, by, the, head, of, an, investment, firm, .>, TokenList<You, wonder, if, he, was, manipulating, the, market, with, his, bombing, targets, .>, TokenList<The, cells, were, operating, in, the, Ghazaliyah, and, al, -

# Generating Representations

We now have our data all set, our models are running and we are good to go!

The next step is now to create the model representations for the sentences in our corpora. Once we have generated these representations we can store them, and train additional diagnostic (/probing) classifiers on top of the representations.

There are a few things you should keep in mind here. Read these carefully, as these tips will save you a lot of time in your implementation.
1. Transformer models make use of Byte-Pair Encodings (BPE), that chunk up a piece of next in subword pieces. For example, a word such as "largely" could be chunked up into "large" and "ly". We are interested in probing linguistic information on the __word__-level. Therefore, we will follow the suggestion of Hewitt et al. (2019a, footnote 4), and create the representation of a word by averaging over the representations of its subwords. So the representation of "largely" becomes the average of that of "large" and "ly".


2. Subword chunks never overlap multiple tokens. In other words, say we have a phrase like "None of the", then the tokenizer might chunk that into "No"+"ne"+" of"+" the", but __not__ into "No"+"ne o"+"f the", as those chunks overlap multiple tokens. This is great for our setup! Otherwise it would have been quite challenging to distribute the representation of a subword over the 2 tokens it belongs to.


3. **Important**: If you closely examine the provided treebank, you will notice that some tokens are split up into multiple pieces, that each have their own POS-tag. For example, in the first sentence the word "Al-Zaman" is split into "Al", "-", and "Zaman". In such cases, the conllu `TokenList` format will add the following attribute: `('misc', OrderedDict([('SpaceAfter', 'No')]))` to these tokens. Your model's tokenizer does not need to adhere to the same tokenization. E.g., "Al-Zaman" could be split into "Al-"+"Za"+"man", making it hard to match the representations with their correct pos-tag. Therefore I recommend you to not tokenize your entire sentence at once, but to do this based on the chunking of the treebank. <br /><br />
Make sure to still incoporate the spaces in a sentence though, as these are part of the BPE of the tokenizer. That is, the tokenizer uses a different token id for `"man"`, than it does for `" man"`: the former could be part of `" woman"`=`" wo`"+`"man"`, whereas the latter would be the used in case *man* occurs at the start of a word. The tokenizer for GPT-2 adds spaces at the start of a token (represented as a `Ġ` symbol). This means that you should keep track whether the previous token had the `SpaceAfter` attribute set to `'No'`: in case it did not, you should manually prepend a `" "` ahead of the token.


4. The LSTM LM does not have the issues related to subwords, but is far more restricted in its vocabulary. Make sure you keep the above points in mind though, when creating the LSTM representations. You might want to write separate functions for the LSTM, but that is up to you.


5. The huggingface transformer models don't return the hidden state by default. To achieve this you can pass `output_hidden_states=True` to a model forward pass. The hidden states are then returned for all intermediate layers as well, the latest entry in this list corresponds to the top layer.


6. **N.B.**: Make sure that when you run a sentence through your model, you do so within a `with torch.no_grad():` block, and that you have run `model.eval()` beforehand as well (to disable dropout).


7. **N.B.**: Make sure to use a token's ``["form"]`` attribute, and not the ``["lemma"]``, as the latter will stem any relevant morphological information from the token. We don't want this, because we want to feed well-formed, grammatical sentences to our model.


I would like to stress that if you feel hindered in any way by the simple code structure that is presented here, you are free to modify it :-) Just make sure it is clear to an outsider what you're doing, some helpful comments never hurt.

In [5]:
# FETCH SENTENCE REPRESENTATIONS
from torch import Tensor
import pickle
from tqdm import tqdm


# Should return a tensor of shape (num_tokens_in_corpus, representation_size)
# Make sure you correctly average the subword representations that belong to 1 token!

def fetch_sen_reps(ud_parses: List[TokenList], model, tokenizer) -> Tensor:
    representation_size = 768
    out = []
    out = torch.empty(1, representation_size)
    index = 0
    
    for sentence in tqdm(ud_parses):
        j = 0
        concat_dict = {}
        token_list = []
        space_after = False
        for token in sentence:
            
            test_var = token["misc"]
            
            if space_after:
                token_return = tokenizer.encode(str(token), add_prefix_space=True)
            else:
                token_return = tokenizer.encode(str(token))
            if test_var:
                space_after = False
            else:
                space_after = True        
            
            len_i = len(token_return)
            concat_dict[j] = [j+i for i in range(len_i)]
            j += len_i
            token_list += token_return
            #(token_list.append(token) for token in token_return)
        token_list = torch.LongTensor(token_list)
        #print(token_list)
        #token_dict["input_ids"] = torch.cat(token_dict["input_ids"], axis=-1)
        #token_dict["attention_mask"] = torch.cat(token_dict["attention_mask"], axis=-1)
        model.eval()
        with torch.no_grad():
            
            out_sentence = model(input_ids = token_list, output_hidden_states=True)
        out_sentence = out_sentence["hidden_states"][-1].squeeze()  
        
        
        out_sent = torch.zeros(len(sentence), representation_size)
        for i, key in enumerate(concat_dict):
            out_sent[i] = torch.mean(out_sentence[concat_dict[key]], axis=0)
        out = torch.cat([out, out_sent])

    return out[1:][:]      


In [6]:
from torch import Tensor
import pickle
from tqdm import tqdm


def fetch_sen_reps_lstm(ud_parses: List[TokenList], model, tokenizer) -> Tensor:
    representation_size = 650
    out = torch.zeros(1, representation_size)
    for sentence in tqdm(ud_parses):
        sent = []
        for token in sentence:
            sent.append(tokenizer[str(token)])
        sent = torch.LongTensor(sent)
        hidden = model.init_hidden(2)
        print(sent.shape)
        with torch.no_grad():
            print(hidden[0].shape)
            model.eval()
            out_sentence = model(sent, hidden[0])

        out = torch.cat([out, out_sentence])
    return out[1:][:]


To validate your activation extraction procedure I have set up the following assertion function as a sanity check. It compares your representation against a pickled version of mine. 

For this I used `distilgpt2`.

In [7]:
def error_msg(model_name, gold_embs, embs, i2w):
    with open(f'{model_name}_tokens1.pickle', 'rb') as f:
        sen_tokens = pickle.load(f)
        
    diff = torch.abs(embs - gold_embs)
    max_diff = torch.max(diff)
    avg_diff = torch.mean(diff)
    
    print(f"{model_name} embeddings don't match!")
    print(f"Max diff.: {max_diff:.4f}\nMean diff. {avg_diff:.4f}")

    print("\nCheck if your tokenization matches with the original tokenization:")
    for idx in sen_tokens.squeeze():
        if isinstance(i2w, list):
            token = i2w[idx]
        else:
            token = i2w.convert_ids_to_tokens(idx.item())
        print(f"{idx:<6} {token}")


def assert_sen_reps(model, tokenizer, lstm, vocab):
    with open('distilgpt2_emb1.pickle', 'rb') as f:
        distilgpt2_emb1 = pickle.load(f)
        
    with open('lstm_emb1.pickle', 'rb') as f:
        lstm_emb1 = pickle.load(f)
    
    corpus = parse_corpus('data/sample/en_ewt-ud-train.conllu')[:1]
    
    own_distilgpt2_emb1 = fetch_sen_reps(corpus, model, tokenizer)
    own_lstm_emb1 = fetch_sen_reps_lstm(corpus, lstm, vocab)
    
    assert distilgpt2_emb1.shape == own_distilgpt2_emb1.shape, \
        f"Distilgpt2 shape mismatch: {distilgpt2_emb1.shape} (gold) vs. {own_distilgpt2_emb1.shape} (yours)"
    assert lstm_emb1.shape == own_lstm_emb1.shape, \
        f"LSTM shape mismatch: {lstm_emb1.shape} (gold) vs. {own_lstm_emb1.shape} (yours)"

    if not torch.allclose(distilgpt2_emb1, own_distilgpt2_emb1, rtol=1e-3, atol=1e-3):
        error_msg("distilgpt2", distilgpt2_emb1, own_distilgpt2_emb1, tokenizer)
    if not torch.allclose(lstm_emb1, own_lstm_emb1, rtol=1e-3, atol=1e-3):
        error_msg("lstm", lstm_emb1, own_lstm_emb1, list(vocab.keys()))


assert_sen_reps(model, tokenizer, lstm, vocab)

  0%|                                                     | 0/1 [00:00<?, ?it/s]

torch.Size([29])
torch.Size([2, 2, 650])


RuntimeError: input must have 3 dimensions, got 2

Next, we should define a function that extracts the corresponding POS labels for each activation, which we do based on the **``"upostag"``** attribute of a token (so not the ``xpostag`` attribute). These labels will be transformed to a tensor containing the label index for each item.

In [8]:
# FETCH POS LABELS
from sklearn import preprocessing

# Should return a tensor of shape (num_tokens_in_corpus,)
# Make sure that when fetching these pos tags for your train/dev/test corpora you share the label vocabulary.
def fetch_pos_tags(ud_parses: List[TokenList], pos_vocab=None) -> Tensor:
    pos_tags = list()
    pos_vocab = defaultdict(lambda:"UNK")
    for sentence in ud_parses:
        for token in sentence:

            pos_tags.append(token["upostag"])

    if pos_vocab:
        targets = pos_vocab.fit_transform(pos_tags)
        pos_tags = torch.as_tensor(targets)
    else:
        pos_vocab = preprocessing.LabelEncoder()
        targets = pos_vocab.fit_transform(pos_tags)
        pos_tags = torch.as_tensor(targets)
  
    return pos_tags, pos_vocab


In [10]:
import os

# Function that combines the previous functions, and creates 2 tensors for a .conllu file: 
# 1 containing the token representations, and 1 containing the (tokenized) pos_tags.

def create_data(filename: str, lm, w2i, pos_vocab=None):
    ud_parses = parse_corpus(filename)
    
    sen_reps = fetch_sen_reps(ud_parses, lm, w2i)
    pos_tags, pos_vocab = fetch_pos_tags(ud_parses, pos_vocab=pos_vocab)
    
    return sen_reps, pos_tags, pos_vocab


lm = model  # or `lstm`
w2i = tokenizer  # or `vocab`
use_sample = True

train_x, train_y, train_vocab = create_data(
    os.path.join('data', 'sample' if use_sample else '', 'en_ewt-ud-train.conllu'),
    lm, 
    w2i
)

dev_x, dev_y, _ = create_data(
    os.path.join('data', 'sample' if use_sample else '', 'en_ewt-ud-dev.conllu'),
    lm, 
    w2i,
    pos_vocab=train_vocab
)

test_x, test_y, _ = create_data(
    os.path.join('data', 'sample' if use_sample else '', 'en_ewt-ud-test.conllu'),
    lm,
    w2i,
    pos_vocab=train_vocab
)

100%|███████████████████████████████████████████| 50/50 [00:03<00:00, 15.99it/s]


# Diagnostic Classification

We now have our models, our data, _and_ our representations all set! Hurray, well done. We can finally move onto the cool stuff, i.e. training the diagnostic classifiers (DCs).

DCs are simple in their complexity on purpose. To read more about why this is the case you could already have a look at the "Designing and Interpreting Probes with Control Tasks" by Hewitt and Liang (esp. Sec. 3.2).

A simple linear classifier will suffice for now, don't bother with adding fancy non-linearities to it.

I am personally a fan of the `skorch` library, that provides `sklearn`-like functionalities for training `torch` models, but you are free to train your dc using whatever method you prefer.

As this is an Artificial Intelligence master and you have all done ML1 + DL, I expect you to use your train/dev/test splits correctly ;-)

In [78]:
# DIAGNOSTIC CLASSIFIER
#import skorch

import torch.nn as nn
import torch.nn.functional as F
import torchmetrics

class LinearClassifier(nn.Module):
  def __init__(self):
    super().__init__()
    self.linear = nn.Linear(768,17)

  def forward(self, x):
    x = self.linear(x)
    x = torch.flatten(x, 1)
    #x = F.softmax(x)
    return x

classifier = LinearClassifier()


parameters = classifier.parameters()
#print(list(parameters))
criterion = torch.nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(classifier.parameters(), lr=0.01)

all_loss = []

accuracy = torchmetrics.Accuracy()
for epoch in tqdm(range(10000)):
     
  output = classifier(train_x)

  loss = criterion(output, train_y)
  #print(loss)
  all_loss.append(loss.item())
  loss.backward()

  optimizer.step()
  optimizer.zero_grad()
    
  if epoch % 1000 == 0 :
    classifier.eval()
    output_dev = classifier(dev_x)
    dev_acc = accuracy(output_dev, dev_y)
    train_acc = accuracy(output, train_y)
    print(f"Validation accuracy on batch {epoch}: {dev_acc}")
    print(f"Training accuracy on batch {epoch}: {train_acc}")
    




  0%|                                       | 29/10000 [00:00<01:13, 135.38it/s]

tensor(5.7740, grad_fn=<NllLossBackward>)
Validation accuracy on batch 0: 0.17924527823925018
Training accuracy on batch 0: 0.018252933397889137
tensor(21.1231, grad_fn=<NllLossBackward>)
tensor(50.6898, grad_fn=<NllLossBackward>)
tensor(70.5067, grad_fn=<NllLossBackward>)
tensor(74.3555, grad_fn=<NllLossBackward>)
tensor(82.7960, grad_fn=<NllLossBackward>)
tensor(60.7454, grad_fn=<NllLossBackward>)
tensor(69.3444, grad_fn=<NllLossBackward>)
tensor(87.4475, grad_fn=<NllLossBackward>)
tensor(99.7461, grad_fn=<NllLossBackward>)
tensor(92.9311, grad_fn=<NllLossBackward>)
tensor(90.4405, grad_fn=<NllLossBackward>)
tensor(104.0150, grad_fn=<NllLossBackward>)
tensor(86.5882, grad_fn=<NllLossBackward>)
tensor(83.2456, grad_fn=<NllLossBackward>)
tensor(67.4583, grad_fn=<NllLossBackward>)
tensor(53.5467, grad_fn=<NllLossBackward>)
tensor(74.0160, grad_fn=<NllLossBackward>)
tensor(93.8645, grad_fn=<NllLossBackward>)
tensor(97.6027, grad_fn=<NllLossBackward>)
tensor(98.8790, grad_fn=<NllLossBackw

  0%|▏                                      | 47/10000 [00:00<01:09, 143.97it/s]

tensor(98.6807, grad_fn=<NllLossBackward>)
tensor(91.9845, grad_fn=<NllLossBackward>)
tensor(115.2502, grad_fn=<NllLossBackward>)
tensor(124.8854, grad_fn=<NllLossBackward>)
tensor(123.1066, grad_fn=<NllLossBackward>)
tensor(98.3102, grad_fn=<NllLossBackward>)
tensor(81.9631, grad_fn=<NllLossBackward>)
tensor(92.1862, grad_fn=<NllLossBackward>)
tensor(114.3640, grad_fn=<NllLossBackward>)
tensor(100.3338, grad_fn=<NllLossBackward>)
tensor(97.5223, grad_fn=<NllLossBackward>)
tensor(94.8409, grad_fn=<NllLossBackward>)
tensor(97.8218, grad_fn=<NllLossBackward>)
tensor(98.2615, grad_fn=<NllLossBackward>)
tensor(100.3355, grad_fn=<NllLossBackward>)
tensor(80.4512, grad_fn=<NllLossBackward>)
tensor(65.1576, grad_fn=<NllLossBackward>)
tensor(63.7601, grad_fn=<NllLossBackward>)
tensor(59.4647, grad_fn=<NllLossBackward>)
tensor(64.1868, grad_fn=<NllLossBackward>)
tensor(85.0068, grad_fn=<NllLossBackward>)
tensor(88.4556, grad_fn=<NllLossBackward>)
tensor(77.9966, grad_fn=<NllLossBackward>)
tenso

  1%|▎                                      | 75/10000 [00:00<01:11, 138.80it/s]

...

 14%|█████▏                               | 1395/10000 [00:06<00:34, 249.31it/s]

tensor(11.9820, grad_fn=<NllLossBackward>)
tensor(12.3129, grad_fn=<NllLossBackward>)
tensor(7.9985, grad_fn=<NllLossBackward>)
tensor(5.1132, grad_fn=<NllLossBackward>)
tensor(7.6060, grad_fn=<NllLossBackward>)
tensor(9.2037, grad_fn=<NllLossBackward>)
tensor(8.3515, grad_fn=<NllLossBackward>)
tensor(9.3004, grad_fn=<NllLossBackward>)
tensor(6.8180, grad_fn=<NllLossBackward>)
tensor(6.2855, grad_fn=<NllLossBackward>)
tensor(8.3076, grad_fn=<NllLossBackward>)
tensor(9.6166, grad_fn=<NllLossBackward>)
tensor(7.3212, grad_fn=<NllLossBackward>)
tensor(5.0829, grad_fn=<NllLossBackward>)
tensor(5.0104, grad_fn=<NllLossBackward>)
tensor(11.6347, grad_fn=<NllLossBackward>)
tensor(11.1719, grad_fn=<NllLossBackward>)
tensor(7.0208, grad_fn=<NllLossBackward>)
tensor(4.3841, grad_fn=<NllLossBackward>)
tensor(6.0862, grad_fn=<NllLossBackward>)
tensor(9.0086, grad_fn=<NllLossBackward>)
tensor(5.9263, grad_fn=<NllLossBackward>)
tensor(4.1457, grad_fn=<NllLossBackward>)
tensor(4.4049, grad_fn=<NllLos

 14%|█████▎                               | 1420/10000 [00:06<00:40, 212.83it/s]


tensor(7.0519, grad_fn=<NllLossBackward>)
tensor(12.5044, grad_fn=<NllLossBackward>)
tensor(11.1054, grad_fn=<NllLossBackward>)
tensor(12.0126, grad_fn=<NllLossBackward>)
tensor(10.8978, grad_fn=<NllLossBackward>)
tensor(10.2884, grad_fn=<NllLossBackward>)
tensor(7.8994, grad_fn=<NllLossBackward>)
tensor(7.6213, grad_fn=<NllLossBackward>)
tensor(7.5544, grad_fn=<NllLossBackward>)
tensor(12.9467, grad_fn=<NllLossBackward>)
tensor(17.6714, grad_fn=<NllLossBackward>)
tensor(8.9055, grad_fn=<NllLossBackward>)
tensor(7.0466, grad_fn=<NllLossBackward>)
tensor(5.4914, grad_fn=<NllLossBackward>)
tensor(8.1565, grad_fn=<NllLossBackward>)
tensor(11.2194, grad_fn=<NllLossBackward>)
tensor(5.5174, grad_fn=<NllLossBackward>)
tensor(6.3788, grad_fn=<NllLossBackward>)
tensor(7.6313, grad_fn=<NllLossBackward>)
tensor(6.8971, grad_fn=<NllLossBackward>)
tensor(10.6711, grad_fn=<NllLossBackward>)
tensor(6.5219, grad_fn=<NllLossBackward>)
tensor(5.0510, grad_fn=<NllLossBackward>)
tensor(7.0571, grad_fn=<N

KeyboardInterrupt: 

In [40]:
classifier.eval()
output_test = classifier(test_x)
test_acc = accuracy(output_test, test_y)
print("The test accuracy is",test_acc.item())

The test accuracy is 0.8360797762870789


# Trees

For our gold labels, we need to recover the node distances from our parse tree. For this we will use the functionality provided by `ete3`, that allows us to compute that directly. I have provided code that transforms a `TokenTree` to a `Tree` in `ete3` format.

In [12]:
# In case you want to transform your conllu tree to an nltk.Tree, for better visualisation

def rec_tokentree_to_nltk(tokentree):
    token = tokentree.token["form"]
    tree_str = f"({token} {' '.join(rec_tokentree_to_nltk(t) for t in tokentree.children)})"

    return tree_str


def tokentree_to_nltk(tokentree):
    from nltk import Tree as NLTKTree

    tree_str = rec_tokentree_to_nltk(tokentree)

    return NLTKTree.fromstring(tree_str)

In [13]:
# !pip install ete3
from ete3 import Tree as EteTree


class FancyTree(EteTree):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, format=1, **kwargs)
        
    def __str__(self):
        return self.get_ascii(show_internal=True)
    
    def __repr__(self):
        return str(self)


def rec_tokentree_to_ete(tokentree):
    idx = str(tokentree.token["id"])
    children = tokentree.children
    if children:
        return f"({','.join(rec_tokentree_to_ete(t) for t in children)}){idx}"
    else:
        return idx
    
def tokentree_to_ete(tokentree):
    newick_str = rec_tokentree_to_ete(tokentree)

    return FancyTree(f"{newick_str};")

In [14]:
# Let's check if it works!
# We can read in a corpus using the code that was already provided, and convert it to an ete3 Tree.

def parse_corpus(filename):
    from conllu import parse_incr

    data_file = open(filename, encoding="utf-8")

    ud_parses = list(parse_incr(data_file))
    
    return ud_parses

corpus = parse_corpus('data/sample/en_ewt-ud-train.conllu')
item = corpus[0]
tokentree = item.to_tree()
ete3_tree = tokentree_to_ete(tokentree)
print(ete3_tree)


   /-2
  |
  |--3
  |
  |--4
  |
  |   /6 /-5
  |  |
  |  |   /-9
  |  |  |
  |  |  |--10
  |  |  |
  |  |  |--11
  |  |-8|
  |  |  |--12
  |-7|  |
  |  |  |--13
  |  |  |
  |  |   \15/-14
-1|  |
  |  |   /-16
  |  |  |
  |  |  |--17
  |  |  |
  |   \18   /-19
  |     |  |
  |     |  |--20
  |     |  |
  |     |  |-23/-22
  |      \21
  |        |--24
  |        |
  |        |   /-25
  |        |  |
  |         \28--26
  |           |
  |            \-27
  |
   \-29


As you can see we label a token by its token id (converted to a string). Based on these id's we are going to retrieve the node distances.

To create the true distances of a parse tree in our treebank, we are going to use the `.get_distance` method that is provided by `ete3`: http://etetoolkit.org/docs/latest/tutorial/tutorial_trees.html#working-with-branch-distances

We will store all these distances in a `torch.Tensor`.

Please fill in the gap in the following method. I recommend you to have a good look at Hewitt's blog post  about these node distances.

In [15]:
def create_gold_distances(corpus):
    all_distances = []
    sen_length = [len(i) for i in corpus]
    max_length = max(sen_length)

    for item in (corpus):
        tokentree = item.to_tree()
        ete_tree = tokentree_to_ete(tokentree)

        sen_len = len(ete_tree.search_nodes())
        distances = torch.full((max_length, max_length), -1)

        for i in range(sen_len):
            for j in range(sen_len):
                node_i = ete_tree&f"{i+1}"
                node_j = ete_tree&f"{j+1}"
                distances[i][j] = node_i.get_distance(node_j)

        all_distances.append(distances)

    return all_distances


The next step is now to do the previous step the other way around. After all, we are mainly interested in predicting the node distances of a sentence, in order to recreate the corresponding parse tree.

Hewitt et al. reconstruct a parse tree based on a _minimum spanning tree_ (MST, https://en.wikipedia.org/wiki/Minimum_spanning_tree). Fortunately for us, we can simply import a method from `scipy` that retrieves this MST.

In [16]:
from scipy.sparse.csgraph import minimum_spanning_tree
import torch


def create_mst(distances):
    distances = torch.triu(distances).detach().numpy()
    
    mst = minimum_spanning_tree(distances).toarray()
    mst[mst>0] = 1.
    
    return mst

Let's have a look at what this looks like, by looking at a relatively short sentence in the sample corpus.

If your addition to the `create_gold_distances` method has been correct, you should be able to run the following snippet. This then shows you the original parse tree, the distances between the nodes, and the MST that is retrieved from these distances. Can you spot the edges in the MST matrix that correspond to the edges in the parse tree?

In [17]:
item = corpus[5]
tokentree = item.to_tree()
ete3_tree = tokentree_to_ete(tokentree)
print(ete3_tree, '\n')

gold_distance = create_gold_distances(corpus[5:6])[0]
print(gold_distance, '\n')

mst = create_mst(gold_distance)
print(mst)


   /2 /-1
  |
  |--3
  |
  |--4
  |
  |   /-6
  |  |
-5|  |--7
  |-8|
  |  |   /-9
  |  |  |
  |   \12--10
  |     |
  |      \-11
  |
   \-13 

tensor([[0, 1, 3, 3, 2, 4, 4, 3, 5, 5, 5, 4, 3],
        [1, 0, 2, 2, 1, 3, 3, 2, 4, 4, 4, 3, 2],
        [3, 2, 0, 2, 1, 3, 3, 2, 4, 4, 4, 3, 2],
        [3, 2, 2, 0, 1, 3, 3, 2, 4, 4, 4, 3, 2],
        [2, 1, 1, 1, 0, 2, 2, 1, 3, 3, 3, 2, 1],
        [4, 3, 3, 3, 2, 0, 2, 1, 3, 3, 3, 2, 3],
        [4, 3, 3, 3, 2, 2, 0, 1, 3, 3, 3, 2, 3],
        [3, 2, 2, 2, 1, 1, 1, 0, 2, 2, 2, 1, 2],
        [5, 4, 4, 4, 3, 3, 3, 2, 0, 2, 2, 1, 4],
        [5, 4, 4, 4, 3, 3, 3, 2, 2, 0, 2, 1, 4],
        [5, 4, 4, 4, 3, 3, 3, 2, 2, 2, 0, 1, 4],
        [4, 3, 3, 3, 2, 2, 2, 1, 1, 1, 1, 0, 3],
        [3, 2, 2, 2, 1, 3, 3, 2, 4, 4, 4, 3, 0]]) 

[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
 [0.

Now that we are able to map edge distances back to parse trees, we can create code for our quantitative evaluation. For this we will use the Undirected Unlabeled Attachment Score (UUAS), which is expressed as:

$$\frac{\text{number of predicted edges that are an edge in the gold parse tree}}{\text{number of edges in the gold parse tree}}$$

To do this, we will need to obtain all the edges from our MST matrix. Note that, since we are using undirected trees, that an edge can be expressed in 2 ways: an edge between node $i$ and node $j$ is denoted by both `mst[i,j] = 1`, or `mst[j,i] = 1`.

You will write code that computes the UUAS score for a matrix of predicted distances, and the corresponding gold distances. I recommend you to split this up into 2 methods: 1 that retrieves the edges that are present in an MST matrix, and one general method that computes the UUAS score.

In [18]:
def edges(mst):
    edges = set()
    n_nodes = mst.shape[0]

    for i in range(n_nodes):
        for j in range(n_nodes):
            if mst[i][j] == 1:
                edges.append((i+1, j+1))

    return edges

def calc_uuas(pred_distances, gold_distances):
    
    gold_mst = create_mst(gold_distances)
    pred_mst = create_mst(pred_distances)
    pred_edges = edges(pred_mst)
    gold_edges = edges(gold_mst)
    pred_in_gold = len(pred_edges.intersection(gold_edges))
    uuas = pred_in_gold/len(gold_distances)
    
    return uuas


# Structural Probes

We now have everything in place to start doing the actual exciting stuff: training our structural probe!
    
To make life easier for you, we will simply take the `torch` code for this probe from John Hewitt's repository. This allows you to focus on the training regime from now on.

In [90]:
import torch.nn as nn
import torch


class StructuralProbe(nn.Module):
    """ Computes squared L2 distance after projection by a matrix.
    For a batch of sentences, computes all n^2 pairs of distances
    for each sentence in the batch.
    """
    def __init__(self, model_dim, rank, device="cpu"):
        super().__init__()
        self.probe_rank = rank
        self.model_dim = model_dim
        
        self.proj = nn.Parameter(data = torch.zeros(self.model_dim, self.probe_rank))
        
        nn.init.uniform_(self.proj, -0.05, 0.05)
        self.to(device)

    def forward(self, batch):
        """ Computes all n^2 pairs of distances after projection
        for each sentence in a batch.
        Note that due to padding, some distances will be non-zero for pads.
        Computes (B(h_i-h_j))^T(B(h_i-h_j)) for all i,j
        Args:
          batch: a batch of word representations of the shape
            (batch_size, max_seq_len, representation_dim)
        Returns:
          A tensor of distances of shape (batch_size, max_seq_len, max_seq_len)
        """
        transformed = torch.matmul(batch, self.proj)
        
        batchlen, seqlen, rank = transformed.size()
        
        transformed = transformed.unsqueeze(2)
        transformed = transformed.expand(-1, -1, seqlen, -1)
        transposed = transformed.transpose(1,2)
        
        diffs = transformed - transposed
        
        squared_diffs = diffs.pow(2)
        squared_distances = torch.sum(squared_diffs, -1)

        return squared_distances

    
class L1DistanceLoss(nn.Module):
    """Custom L1 loss for distance matrices."""
    def __init__(self):
        super().__init__()

    def forward(self, predictions, label_batch, length_batch):
        """ Computes L1 loss on distance matrices.
        Ignores all entries where label_batch=-1
        Normalizes first within sentences (by dividing by the square of the sentence length)
        and then across the batch.
        Args:
          predictions: A pytorch batch of predicted distances
          label_batch: A pytorch batch of true distances
          length_batch: A pytorch batch of sentence lengths
        Returns:
          A tuple of:
            batch_loss: average loss in the batch
            total_sents: number of sentences in the batch
        """
        labels_1s = (label_batch != -1).float()
        predictions_masked = predictions * labels_1s
        labels_masked = label_batch * labels_1s
        total_sents = torch.sum((length_batch != 0)).float()
        squared_lengths = length_batch.pow(2).float()

        if total_sents > 0:
            loss_per_sent = torch.sum(torch.abs(predictions_masked - labels_masked), dim=(1,2))
            #print("This is loss per sentence shape", loss_per_sent.shape)
            #print("This is squared length shape", squared_lengths.shape)
            normalized_loss_per_sent = loss_per_sent / squared_lengths
            batch_loss = torch.sum(normalized_loss_per_sent) / total_sents
        
        else:
            batch_loss = torch.tensor(0.0)
        
        return batch_loss, total_sents


In [26]:
def fetch_sen_reps_tree(ud_parses: List[TokenList], model, tokenizer, concat=False) -> Tensor:
    representation_size = 768
    out = []
    index = 0
    sen_length = [len(i) for i in ud_parses]
    max_length = max(sen_length)
    
    for sentence in tqdm(ud_parses):
        j = 0
        concat_dict = {}
        token_list = []
        space_after = False
        for token in sentence:
            
            test_var = token["misc"]
            
            if space_after:
                token_return = tokenizer.encode(str(token), add_prefix_space=True)
            else:
                token_return = tokenizer.encode(str(token))
            if test_var:
                space_after = False
            else:
                space_after = True        
            
            len_i = len(token_return)
            concat_dict[j] = [j+i for i in range(len_i)]
            j += len_i
            token_list += token_return
        token_list = torch.LongTensor(token_list)
        model.eval()
        with torch.no_grad():
            
            out_sentence = model(input_ids = token_list, output_hidden_states=True)
        out_sentence = out_sentence["hidden_states"][-1].squeeze()  
        
        
        out_sent = torch.zeros(max_length, representation_size)
        for i, key in enumerate(concat_dict):
            out_sent[i] = torch.mean(out_sentence[concat_dict[key]], axis=0)
        out.append(out_sent)
    
    out = torch.stack(out)
    return out

In [27]:
fetch_sen_reps_tree(ud_parses[:2], model, tokenizer)

100%|█████████████████████████████████████████████| 2/2 [00:00<00:00, 14.40it/s]


tensor([[[-0.0792,  0.3346, -0.1135,  ..., -0.2124,  0.1306, -0.1681],
         [-0.3783,  0.0423, -0.0740,  ..., -0.0850,  0.4072, -0.2320],
         [-0.4129,  0.4578, -0.7827,  ...,  0.0322, -0.5836,  0.4090],
         ...,
         [ 0.7832,  0.8050,  0.0341,  ..., -0.1987, -0.6397,  0.0914],
         [ 0.5115,  0.4130, -0.8091,  ...,  0.1806, -0.1424,  0.2259],
         [-0.8412,  0.2163,  0.0278,  ..., -0.1575, -0.1994, -0.0057]],

        [[-0.1087,  0.3302, -0.0595,  ..., -0.1555,  0.1397, -0.1980],
         [ 0.5315, -0.1347,  0.6371,  ...,  0.4578,  0.1667,  0.2503],
         [ 0.3531,  0.0395, -0.6217,  ..., -0.0975,  0.2088,  0.0475],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]]])

I have provided a rough outline for the training regime that you can use. Note that the hyper parameters that I provide here only serve as an indication, but should be (briefly) explored by yourself.

As can be seen in Hewitt's code above, there exists functionality in the probe to deal with batched input. It is up to you to use that: a (less efficient) method can still incorporate batches by doing multiple forward passes for a batch and computing the backward pass only once for the summed losses of all these forward passes. (_I know, this is not the way to go, but in the interest of time that is allowed ;-), the purpose of the assignment is writing a good paper after all_).

In [93]:
from torch import optim
from tqdm import tqdm

'''
Similar to the `create_data` method of the previous notebook, I recommend you to use a method 
that initialises all the data of a corpus. Note that for your embeddings you can use the 
`fetch_sen_reps` method again. However, for the POS probe you concatenated all these representations into 
1 big tensor of shape (num_tokens_in_corpus, model_dim). 

The StructuralProbe expects its input to contain all the representations of 1 sentence, so I recommend you
to update your `fetch_sen_reps` method in a way that it is easy to retrieve all the representations that 
correspond to a single sentence.
''' 

def init_corpus(path, concat=False, cutoff=None):
    """ Initialises the data of a corpus.
    
    Parameters
    ----------
    path : str
        Path to corpus location
    concat : bool, optional
        Optional toggle to concatenate all the tensors
        returned by `fetch_sen_reps`.
    cutoff : int, optional
        Optional integer to "cutoff" the data in the corpus.
        This allows only a subset to be used, alleviating 
        memory usage.
    """
    corpus = parse_corpus(path)[:cutoff]

    embs = fetch_sen_reps_tree(corpus, model, tokenizer, concat=concat)    
    gold_distances = torch.stack(create_gold_distances(corpus))
    
    return embs, gold_distances


# I recommend you to write a method that can evaluate the UUAS & loss score for the dev (& test) corpus.
# Feel free to alter the signature of this method.
def evaluate_probe(probe, _data):
    probe.eval()
    x, y = _data
    loss_function =  L1DistanceLoss()
    output = probe(x)
    loss_score = loss_function(output, y)
    uuas_score = calc_uuas(output, y)
    
    
    return loss_score, uuas_score


# Feel free to alter the signature of this method.
def train(_data):
    emb_dim = 768
    rank = 64
    lr = 10e-4
    batch_size = 24
    epochs = 100

    probe = StructuralProbe(emb_dim, rank)
    optimizer = optim.Adam(probe.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5,patience=1)
    loss_function =  L1DistanceLoss()
    x, y = _data
    dev_losses = []
    dev_uuass = []

    for epoch in tqdm(range(epochs)):

        for i in range(0, len(corpus), batch_size):
            x_batch, y_batch = x[i:i+batch_size], y[i:i+batch_size]
            optimizer.zero_grad()

            output = probe(x_batch)
            #print(x_batch)
            length_batch = torch.count_nonzero(x_batch, dim=1)
            batch_loss, _ = loss_function(output, y_batch, length_batch[:,0])

            batch_loss.backward()
            optimizer.step()
        
        dev_loss, dev_uuas = evaluate_probe(probe, _dev_data)
        dev_losses.append(dev_loss)
        dev_uuass.append(dev_uuas)

        # Using a scheduler is up to you, and might require some hyper param fine-tuning
        scheduler.step(dev_loss)

    test_loss, test_uuas = evaluate_probe(probe, _test_data)
    return dev_losses, dev_uuas, test_loss, test_uuas

In [35]:
import os

train_data = init_corpus(os.path.join('', 'data/en_ewt-ud-train.conllu'), cutoff=300)
_dev_data = init_corpus(os.path.join('', 'data/en_ewt-ud-train.conllu'), cutoff=100)
_test_data = init_corpus(os.path.join('', 'data/en_ewt-ud-train.conllu'), cutoff=100)

100%|█████████████████████████████████████████| 100/100 [00:06<00:00, 15.60it/s]


In [94]:
dev_losses, dev_uuas, test_loss, test_uuas = train(train_data)

  0%|                                                   | 0/100 [00:00<?, ?it/s]


TypeError: forward() missing 1 required positional argument: 'length_batch'

## LaTeX trees

For your report you might want to add some of those fancy dependency tree plots like those of Figure 2 in the Structural Probing paper. For that you can use the following code, that outputs the corresponding LaTeX markup.

**N.B.**: for the latex tikz tree the first token in a sentence has index 1 (instead of 0), so take that into account with the predicted and gold edges that you pass to the method.

In [67]:
def print_tikz(predicted_edges, gold_edges, words):
    """ Turns edge sets on word (nodes) into tikz dependency LaTeX.
    Parameters
    ----------
    predicted_edges : Set[Tuple[int, int]]
        Set (or list) of edge tuples, as predicted by your probe.
    gold_edges : Set[Tuple[int, int]]
        Set (or list) of gold edge tuples, as obtained from the treebank.
    words : List[str]
        List of strings representing the tokens in the sentence.
    """

    string = """\\begin{dependency}[hide label, edge unit distance=.5ex]
    \\begin{deptext}[column sep=0.05cm]
    """

    string += (
        "\\& ".join([x.replace("$", "\$").replace("&", "+") for x in words])
        + " \\\\\n"
    )
    string += "\\end{deptext}" + "\n"
    for i_index, j_index in gold_edges:
        string += "\\depedge[-]{{{}}}{{{}}}{{{}}}\n".format(i_index, j_index, ".")
    for i_index, j_index in predicted_edges:
        string += f"\\depedge[-,edge style={{red!60!}}, edge below]{{{i_index}}}{{{j_index}}}{{.}}\n"
    string += "\\end{dependency}\n"
    print(string)